In [1]:
from collections import Counter
from itertools import count

from heapq import heappush, heappop
from collections import defaultdict

from extratools.settools import addtoset

from functools import partial
import re

from extratools.strtools import str2grams
from extratools.dicttools import invertedindex

In [9]:
# 正则表达式，\W:不匹配大小写字母下划线,再加上下划线
reSplit = re.compile(r"[_\W]+", re.U)
print(reSplit.split('a_a'))

['a', 'a']


In [14]:
def str2words(s):
    return [w for w in reSplit.split(s) if len(w) > 0]

In [18]:
# 创建str2int的字典，按照出现的频率排序
def createGramMap(sRawStrSets):
    gramFreqs: Counter = Counter()

    for sRawStrSet in sRawStrSets:
        gramFreqs.update(sRawStrSet)

    gramMapper = count(start=0)

    return {
        gram: next(gramMapper)
        for gram, _ in sorted(
            gramFreqs.items(),
            key=lambda x: (x[1], x[0])
        )
    }
a = [['a','b','c'],['a','a','a']]
a_map = createGramMap(a)
print(a_map)

{'b': 0, 'c': 1, 'a': 2}


In [19]:
def updateGramMap(gramMap,rRawStrSet):
    gramMapper = count(start=-1, step=-1)

    gramMap.update(
        (gram, next(gramMapper))
        for gram in rRawStrSet
        if gram not in gramMap
    )
b = ['c','d','e']
updateGramMap(a_map,b)
print(a_map)

{'b': 0, 'c': 1, 'a': 2, 'd': -1, 'e': -2}


In [21]:
def applyGramMap(gramMap,rRawStrSet):
    return [gramMap[gram] for gram in rRawStrSet]
#     return sorted(gramMap[gram] for gram in rRawStrSet)
a_result = applyGramMap(a_map,['a','b','c'])
print(a_result)

[2, 0, 1]


In [22]:
import Levenshtein

In [29]:
d = Levenshtein.distance(u'我吃大米饭',u'我吃大米')
print(d)
print(1- d/len(u'我吃大米饭'))

1
0.8


In [ ]:
d = Levenshtein.distance(u'我爱吃大米饭',u'我爱吃大米')

In [30]:
def edit_distance(a,b):
    return safediv(Levenshtein.distance(a,b)/len(a))

In [31]:
def findBest(
        rStr: StringSet,
        sStrs: List[StringSet], sIndex: Index,
        k: int, tie: bool,
        upBoundFunc: Callable[[int, int, int, int, int], float]
    ) -> Output:
    worstSim = 0.0
    totalNum = 0

    simHeap: List[float] = []
    simMap: Dict[float, List[int]] = defaultdict(list)

    lnSet: Set[int] = set()
    for i, item in enumerate(rStr):
        if upBoundFunc(len(rStr), i + 1, len(rStr) - (i + 1) + 1, 1, 1) < worstSim:
            break

        for ln, p in sIndex[item]:
            if not addtoset(lnSet, ln):
                continue

            currSim = checkSim(
                worstSim, upBoundFunc,
                rStr, sStrs[ln], i + 1, p + 1, 1
            )
            if currSim is None:
                continue

            if currSim not in simMap:
                heappush(simHeap, currSim)
            simMap[currSim].append(ln)
            totalNum += 1

            if totalNum > k:
                currWorstSim = simHeap[0]
                currWorstNum = len(simMap[currWorstSim])

                if totalNum - currWorstNum >= k:
                    del simMap[currWorstSim]
                    totalNum -= currWorstNum
                    heappop(simHeap)
                elif not tie:
                    del simMap[currWorstSim][currWorstNum - (totalNum - k):]
                    totalNum = k

            if totalNum >= k:
                worstSim = simHeap[0]

    return sorted(simMap.items(), key=lambda x: x[0], reverse=True)

NameError: name 'StringSet' is not defined